In [1]:
import pandas as pd
import numpy as np
from functools import reduce

In [11]:
# 1. Extract time-invariant from the most recent year
col_geo = ["UNITID", "INSTNM", "ADDR", "CITY", "STABBR", "LONGITUD", "LATITUDE"]
col_adm = ["UNITID"] + [f"ADMCON{n}" for n in range(1, 10)]
col_fee = ["UNITID", "TUITION2", "TUITION3"]

geo = pd.read_csv("./University_data/hd2019.csv", encoding="ISO-8859-1").loc[:, col_geo]
adm = pd.read_csv("./University_data/adm2019.csv").loc[:, col_adm]
fee = pd.read_csv("./University_data/ic2019_ay.csv").loc[:, col_fee].replace(".", np.nan).dropna().astype(int)

# 2. Calculate gender-specific admission rates in the past 10 years
col_gen = ["UNITID", "APPLCNM", "ADMSSNM", "APPLCNW", "ADMSSNW"]
adm_gender = pd.DataFrame()
for i in range(2014, 2020):
    gen = pd.read_csv(f"./University_data/adm{i}.csv").loc[:, col_gen]
    gen["ADMRM"] = gen["ADMSSNM"] / gen["APPLCNM"]
    gen["ADMRW"] = gen["ADMSSNW"] / gen["APPLCNW"]
    adm_gender = adm_gender.append(gen)
    
adm_gender = adm_gender.groupby("UNITID")[["ADMRM", "ADMRW"]].mean().dropna().round(4)

# 3. Calculate SAT/ACT scores
col_sco = ["UNITID", "SATVR25", "SATVR75", "SATMT25", "SATMT75", 
           "ACTCM25", "ACTCM75", "ACTEN25", "ACTEN75", "ACTMT25", "ACTMT75"]
adm_sco = pd.DataFrame()
for i in range(2014, 2020):
    sco = pd.read_csv(f"./University_data/adm{i}.csv")
    sco.columns = [col.strip() for col in sco.columns]
    sco = sco.loc[:, col_sco]
    adm_sco = adm_sco.append(sco)
    
adm_sco = adm_sco.groupby("UNITID").mean().dropna().round(4)

# 4. Merge
df = reduce(lambda left, right: pd.merge(left, right, how="inner", on="UNITID"), 
            [geo, adm, fee, adm_gender, adm_sco]).set_index("UNITID")

In [12]:
# Check the school with annual tuition more than 50000
df[df["TUITION3"] > 50000].drop_duplicates(subset="INSTNM")

,INSTNM,ADDR,CITY,STABBR,LONGITUD,LATITUDE,ADMCON1,ADMCON2,ADMCON3,ADMCON4,...,SATVR25,SATVR75,SATMT25,SATMT75,ACTCM25,ACTCM75,ACTEN25,ACTEN75,ACTMT25,ACTMT75
UNITID,,,,,,,,,,,,,,,,,,,,,
110404,California Institute of Technology,1200 E California Blvd,Pasadena,CA,-118.125878,34.137349,2,2,1,1,...,738.3333,788.3333,778.3333,800.0000,34.3333,35.5000,34.3333,35.6667,34.8333,36.0000
111948,Chapman University,One University Dr.,Orange,CA,-117.852518,33.793020,1,2,1,1,...,575.0000,661.6667,573.3333,670.0000,25.0000,30.0000,25.1667,32.3333,24.0000,28.8333
112260,Claremont McKenna College,500 E 9th St,Claremont,CA,-117.711188,34.102154,2,2,1,1,...,663.3333,740.0000,680.0000,770.0000,30.3333,33.5000,30.6667,34.3333,28.5000,33.6667
115409,Harvey Mudd College,301 Platt Blvd.,Claremont,CA,-117.709837,34.106515,5,5,1,1,...,696.3333,768.3333,751.6667,800.0000,32.8333,35.0000,33.1667,35.5000,33.1667,35.1667
120254,Occidental College,1600 Campus Rd,Los Angeles,CA,-118.210908,34.127124,5,5,1,2,...,623.3333,706.6667,620.0000,720.0000,27.6667,31.5000,28.8333,34.0000,26.0000,30.8333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237057,Whitman College,345 Boyer Ave,Walla Walla,WA,-118.331546,46.069815,5,5,1,2,...,601.6667,711.6667,600.0000,713.3333,27.6667,32.1667,28.0000,35.0000,25.0000,31.0000
238333,Beloit College,700 College St,Beloit,WI,-89.029285,42.504022,1,5,1,2,...,546.6667,676.6667,541.6667,693.3333,23.0000,29.6667,23.3333,31.3333,21.3333,27.6667
243744,Stanford University,,Stanford,CA,-122.167359,37.429434,2,3,1,2,...,690.0000,773.3333,710.0000,795.0000,31.5000,34.8333,32.8333,35.5000,30.0000,35.0000


In [13]:
# Check info by specific schools
df[df["INSTNM"] == "University of Pennsylvania"]

,INSTNM,ADDR,CITY,STABBR,LONGITUD,LATITUDE,ADMCON1,ADMCON2,ADMCON3,ADMCON4,...,SATVR25,SATVR75,SATMT25,SATMT75,ACTCM25,ACTCM75,ACTEN25,ACTEN75,ACTMT25,ACTMT75
UNITID,,,,,,,,,,,,,,,,,,,,,
215062,University of Pennsylvania,34th & Spruce Street,Philadelphia,PA,-75.19391,39.950929,1,3,1,2,...,686.6667,765.0,715.0,791.6667,31.8333,34.6667,32.8333,35.3333,30.5,34.8333


In [14]:
# Export data
df.to_csv("./University_data/merged_data.csv")